In [8]:
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing

In [3]:
def error_relativo(x,medido):
    return abs(x - medido) * 100 / x

In [32]:
propiedades = pd.read_csv('/home/mati/Desktop/properati_descripciones_knn.csv')

In [33]:
set_entrenamiento = propiedades.loc[(propiedades.Year < 2017) | (propiedades.Month < 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd','seguridad','gimnasio']]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd','seguridad','gimnasio']].sample(15000)

In [34]:
set_entrenamiento_datos = set_entrenamiento.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                                                   'seguridad','gimnasio']]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [35]:
randomforest = RandomForestRegressor()
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                            'seguridad','gimnasio']].apply(lambda x: randomforest.predict(x)[0],axis = 1)
set_pruebas.loc[:,'error_relativo'] = set_pruebas.loc[:,['price_aprox_usd','resultado']].\
                apply(lambda x: error_relativo(x[0],x[1]), axis = 1)

In [36]:
print(randomforest.score(set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                            'seguridad','gimnasio']],set_pruebas.loc[:,'price_aprox_usd']) * 100)
set_pruebas[set_pruebas.error_relativo < 200].error_relativo.describe()

66.8367541021


count    14883.000000
mean        15.374766
std         21.768486
min          0.000000
25%          1.785450
50%          8.166667
75%         20.225245
max        199.153133
Name: error_relativo, dtype: float64

## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [14]:
propiedades = pd.read_csv('/home/mati/Desktop/properati_descripciones_knn.csv')

set_entrenamiento = propiedades.loc[(propiedades.Year < 2017) | (propiedades.Month < 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd','seguridad','gimnasio']]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd','seguridad','gimnasio']].sample(15000)

set_entrenamiento_datos = set_entrenamiento.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                                                   'seguridad','gimnasio']]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [ ]:
res = []
lista_arboles = [5,10,20,30,100]
lista_feature = [0.5]

for cant_arbol in lista_arboles:
    for feature in lista_feature:
        randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
        randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                            'seguridad','gimnasio']].apply(lambda x: randomforest.predict(x)[0],axis = 1)
        set_pruebas.loc[:,'error_relativo'] = set_pruebas.loc[:,['price_aprox_usd','resultado']].\
                apply(lambda x: error_relativo(x[0],x[1]), axis = 1)
        error = set_pruebas.loc[set_pruebas.error_relativo < 200,:].error_relativo.mean()
        cant = set_pruebas.loc[set_pruebas.error_relativo < 200,:].error_relativo.count()
        precision = randomforest.score(set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                            'seguridad','gimnasio']],set_pruebas.loc[:,'price_aprox_usd']) * 100
        res.append((cant_arbol,feature,error,cant,precision))
        print(cant_arbol,' - ',feature)

In [21]:
for r in res:
    print("Cant arboles = {}, % features = {}, error = {}, cantidad = {}, precision = {:.2f}".\
          format(r[0],r[1],r[2],r[3],r[4]))

Cant arboles = 5, % features = 0.5, error = 15.51140013061924, cantidad = 14859, precision = 85.35
Cant arboles = 5, % features = 0.7, error = 15.50905961189723, cantidad = 14854, precision = 64.52
Cant arboles = 5, % features = 1.0, error = 15.371427728972733, cantidad = 14847, precision = 63.64
Cant arboles = 10, % features = 0.5, error = 15.491710897184387, cantidad = 14859, precision = 85.61
Cant arboles = 10, % features = 0.7, error = 15.470925951849924, cantidad = 14858, precision = 80.35
Cant arboles = 10, % features = 1.0, error = 15.421936805593825, cantidad = 14854, precision = 85.20
Cant arboles = 20, % features = 0.5, error = 15.391098003460806, cantidad = 14857, precision = 80.54
Cant arboles = 20, % features = 0.7, error = 15.429889570567612, cantidad = 14851, precision = 70.31
Cant arboles = 20, % features = 1.0, error = 15.381341070970887, cantidad = 14853, precision = 60.37
Cant arboles = 30, % features = 0.5, error = 15.451073242401757, cantidad = 14858, precision = 8

## Tomamos features = 0.5 y buscamos cant entre 5 y 15

In [25]:
lista_arboles = [9]
lista_feature = [0.6,0.7,0.8,0.9,1.0]

for cant_arbol in lista_arboles:
    for feature in lista_feature:
        randomforest = RandomForestRegressor(n_estimators = cant_arbol, max_features = feature)
        randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                            'seguridad','gimnasio']].apply(lambda x: randomforest.predict(x)[0],axis = 1)
        set_pruebas.loc[:,'error_relativo'] = set_pruebas.loc[:,['price_aprox_usd','resultado']].\
                apply(lambda x: error_relativo(x[0],x[1]), axis = 1)
        error = set_pruebas.loc[set_pruebas.error_relativo < 200,:].error_relativo.mean()
        cant = set_pruebas.loc[set_pruebas.error_relativo < 200,:].error_relativo.count()
        precision = randomforest.score(set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                            'seguridad','gimnasio']],set_pruebas.loc[:,'price_aprox_usd']) * 100
        res.append((cant_arbol,feature,error,cant,precision))
        print(cant_arbol,' - ',feature)

9  -  0.6
9  -  0.7
9  -  0.8
9  -  0.9
9  -  1.0


In [27]:
for r in res:
    print("Cant arboles = {}, % features = {}, error = {:.2f}, cantidad = {}, precision = {:.2f}".\
          format(r[0],r[1],r[2],r[3],r[4]))

Cant arboles = 5, % features = 0.5, error = 15.46, cantidad = 14854, precision = 84.18
Cant arboles = 6, % features = 0.5, error = 15.37, cantidad = 14852, precision = 70.09
Cant arboles = 7, % features = 0.5, error = 15.30, cantidad = 14854, precision = 82.35
Cant arboles = 8, % features = 0.5, error = 15.38, cantidad = 14854, precision = 83.46
Cant arboles = 9, % features = 0.5, error = 15.39, cantidad = 14857, precision = 85.64
Cant arboles = 10, % features = 0.5, error = 15.46, cantidad = 14851, precision = 80.89
Cant arboles = 11, % features = 0.5, error = 15.47, cantidad = 14862, precision = 84.11
Cant arboles = 12, % features = 0.5, error = 15.43, cantidad = 14850, precision = 85.06
Cant arboles = 13, % features = 0.5, error = 15.36, cantidad = 14851, precision = 84.08
Cant arboles = 14, % features = 0.5, error = 15.39, cantidad = 14854, precision = 85.40
Cant arboles = 15, % features = 0.5, error = 15.45, cantidad = 14856, precision = 80.13
Cant arboles = 9, % features = 0.6, e

## Tomamos cant = 9, features = 0.7

In [41]:
propiedades = pd.read_csv('/home/mati/Desktop/properati_descripciones_knn.csv')

set_entrenamiento = propiedades.loc[(propiedades.Year < 2017) | (propiedades.Month < 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd','seguridad','gimnasio']]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),\
        ['superficie','place_name_encoded','property_type_encoded','price_aprox_usd','seguridad','gimnasio']].sample(15000)

set_entrenamiento_datos = set_entrenamiento.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                                                   'seguridad','gimnasio']]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [42]:
randomforest = RandomForestRegressor(n_estimators = 9, max_features = 0.7)
randomforest.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                            'seguridad','gimnasio']].apply(lambda x: randomforest.predict(x)[0],axis = 1)
set_pruebas.loc[:,'error_relativo'] = set_pruebas.loc[:,['price_aprox_usd','resultado']].\
                apply(lambda x: error_relativo(x[0],x[1]), axis = 1)
        
precision = randomforest.score(set_pruebas.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                            'seguridad','gimnasio']],set_pruebas.loc[:,'price_aprox_usd']) * 100

print(precision)

86.0141632861


# calculo con los verdaderos datos a analizar

In [44]:
analizar = pd.read_csv("/home/mati/Desktop/properati_dataset_modificado.csv")

In [45]:
analizar.loc[:,'price_usd'] = analizar.loc[:,['superficie','place_name_encoded','property_type_encoded',\
                            'seguridad','gimnasio']].apply(lambda x: randomforest.predict(x)[0],axis = 1)

In [46]:
resultado = analizar.loc[:,['id','price_usd']]
resultado.to_csv('Random_Forests_resultados', index = False)
resultado.head()

,id,price_usd
0,3632,230714.056167
1,3633,145322.962963
2,2263404,108566.910048
3,2263405,70591.448802
4,2263406,57015.873016
